In [2]:
import cv2
import numpy as np
import joblib
from skimage.feature import local_binary_pattern, hog

In [3]:
# ========== تحميل الأدوات ==========
scaler = joblib.load("scaler.pkl")
pca = joblib.load("pca.pkl")
svm_model = joblib.load("svm_model.pkl")

In [4]:
# ========== دوال استخراج الميزات ==========
def extract_lbp_features(images, P=8, R=1):
    features = []
    for img in images:
        lbp = local_binary_pattern(img, P, R, method="uniform")
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, P+3), range=(0, P+2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        features.append(hist)
    return np.array(features)

def extract_hog_features(images):
    features = []
    for img in images:
        hog_features, _ = hog(img, pixels_per_cell=(16, 16),
                              cells_per_block=(2, 2),
                              visualize=True, feature_vector=True)
        features.append(hog_features)
    return np.array(features)

def preprocess_image(img_path, img_size=(224,224)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, img_size)

    # تحويل إلى شكل [1, H, W]
    X = np.array([img])

    # استخراج الميزات
    X_lbp = extract_lbp_features(X)
    X_hog = extract_hog_features(X)

    # دمج
    X_features = np.hstack([X_lbp, X_hog])

    # scaling + pca
    X_scaled = scaler.transform(X_features)
    X_pca = pca.transform(X_scaled)

    return X_pca

In [5]:
# ========== التنبؤ ==========
img_path = input("Enter image path: ").strip()

features = preprocess_image(img_path)
y_pred = svm_model.predict(features)
y_prob = svm_model.predict_proba(features)[0]

label = "(MS)" if y_pred[0] == 1 else "(Healthy)"

print("\nPrediction Result:")
print(f"Diagnosis: {label}")
print(f"Confidence: {y_prob[y_pred[0]]*100:.2f}%")



Prediction Result:
Diagnosis: (Healthy)
Confidence: 85.53%
